<h3> 1) Determine the end-to-end transmission duration for a packet. <h3>

In [11]:
#Propagation delay in ms
t_prop_1 = 2
t_prop_2 = 4 
t_prop_3 = 10 
t_prop_4 = 1

#Capacity in bits
capacity_1 = 3_000_000
capacity_2 = 9_000_000
capacity_3 = 2_000_000
capacity_4 = 2_000_000

#Packetsize in bits
packet_size = 1125 * 8

1.$t_{tx} = L/C$ <br>
2.$t_E2E_1 = 
\sum_{n=1}^{4} t_{tx_i} +
\sum_{n=1}^{4} t_{prop_i}
$

In [12]:

#Calculate transmissons delay
t_tx_1 = packet_size / capacity_1 * 1000
t_tx_2 = packet_size / capacity_2 * 1000
t_tx_3 = packet_size / capacity_3 * 1000
t_tx_4 = packet_size / capacity_4 * 1000



#Calculate end-to-end transmisson for one packet
t_E2E_1 = 0
for i in range(1,5):
     # Erstelle den Variablennamen mit f-string-Formatierung
    t_prop_i = f"t_prop_{i}"
    t_tx_i = f"t_tx_{i}"
    print("transmission delay: " + str(globals()[t_tx_i]) + "ms")
    
    # Rufe die Variable auf (dies ist nur ein Beispiel, du kannst hier deine eigene Logik hinzufügen)
    t_E2E_1 =+ globals()[t_prop_i]
    t_E2E_1 =+ globals()[t_tx_i]

print("End-to-End-Delay: " + str(t_E2E_1) + "ms")

transmission delay: 3.0ms
transmission delay: 1.0ms
transmission delay: 4.5ms
transmission delay: 4.5ms
End-to-End-Delay: 4.5ms


<h3> 2) The source sends packets at a rate of 500 packets per second. Determine for each link which packets are lost. Consider the first 20 packets sent for this. <h3>

In [13]:
buffer_size_1 = 6*8000
buffer_size_2 = 5*8000
buffer_size_3 = 4*8000
buffer_size_4 = 3*8000

import math

print("Buffer 1: " + str(math.floor(buffer_size_1/packet_size)) + " packets")
print("Buffer 2: " + str(math.floor(buffer_size_2/packet_size)) + " packets")
print("Buffer 3: " + str(math.floor(buffer_size_3/packet_size)) + " packets")
print("Buffer 4: " + str(math.floor(buffer_size_4/packet_size)) + " packets")

Buffer 1: 5 packets
Buffer 2: 4 packets
Buffer 3: 3 packets
Buffer 4: 2 packets


**Link2**<br>
*arrival-interval:* 0.0ms<br>
*transmission delay:* 3.0ms

-> Since all 20 pakets arrive at the same time, only **5** are buffered! (As many as the buffer fits)<br>
-> Pakets remaining: 5

**Link2**<br>
*arrival-interval:* 3.0ms<br>
*transmission delay:* 1.0ms

| Zeitpunkt (ms) | Ereignis         | Buffer               | Übertragung | Ende        |
|:--------------:|:----------------:|:--------------------:|:-----------:|:-----------:|
| 0.0            | A1               |                      | P1          | 1.0         |
| 1.0            | D1               |                      |             |             |
| 3.0            | A2               |                      | P2          | 4.0         |
| 4.0            | D2               |                      |             |             |
| ...            | ...              | ...                  | ...         | ...         |


-> transmission-delay is lower then arrival-interval: No pakets are lost<br>
-> Pakets remaining: 5

**Link2**<br>
*arrival-interval:* 1.0ms<br>
*transmission delay:* 4.5ms

| Zeitpunkt (ms) | Ereignis         | Buffer               | Übertragung | Ende        |
|:--------------:|:----------------:|:--------------------:|:-----------:|:-----------:|
| 0.0            | A1               |                      | P1          | 4.5         |
| 1.0            | A2               | A2                   |             |             |
| 2.0            | A3               | A3,A2                |             |             |
| 3.0            | A4               | A4,A3,A2             |             |             |
| 4.0            | A5(dropped)      | A4,A3,A2             |             |             |
| 4.5            | D1               | A4,A3                | P2          | 9.0         |
| ...            | ...              | ...                  | ...         | ...         |

-> One Paket(A5) dropped! <br>
-> Pakets remaining: 4

**Link2**<br>
*arrival-interval:* 4.5ms<br>
*transmission delay:* 4.5ms

-> since arrival-interval and transmission-delay are the same no pakets are lost<br>
-> 4 pakets remaining

**We have a 75% package loss**

<h4> 
3) To reduce the number of packet losses, a Go-Back-N protocol with a transmission window of N=15 packets is used at the transport layer. The timeout is set to 100ms, and the transmission of an ACK from Z to Q takes 15ms. Determine the arrival time of the last packet when the source finishes transmitting after 20 packets. Furthermore, assume that the source transmits packets at a rate of 500 packets per second.<h4>

So we know that only 4 pakets will go trough in one go <br>

The time for all 4 pakets to arrive is: 

**Formel**

1. $TE2E(n) = TE2E(1) + (n - 1) * L_{\text{min}}(C_1)$






In [14]:
t_E2E_4 = round(t_E2E_1 + ((4-1) * t_tx_4), 3)
print("End-to-End-Delay for 4 packets: " + str(t_E2E_4) + "ms")

End-to-End-Delay for 4 packets: 18.0ms


With this information we can determine the Time that all 20 packets need for transmitting: <br>

| Event                 | Time Delay | Cumulative Time |
|-----------------------|------------|-----------------|
| Sending packets 1-15  | 0ms        | 0ms             |
| Arrival packets 1-4   | 18ms       | 18ms            |
| ACK of 4th packet     | 15ms       | 33ms            |
| Timeout for packet 5  | 100ms      | 133ms           |
| Sending packets 5-19  | 0ms        | 133ms           |
| Arrival packets 5-8   | 18ms       | 151ms           |
| ACK of 8th packet     | 15ms       | 166ms           |
| Timeout for packet 9  | 100ms      | 266ms           |
| Sending packets 9-20  | 0ms        | 266ms           |
| Arrival packets 9-12  | 18ms       | 284ms           |
| ACK of 12th packet    | 15ms       | 299ms           |
| Sending packets 13-20 | 0ms        | 299ms           |
| Arrival packets 13-16 | 18ms       | 317ms           |
| ACK of 16th packet    | 15ms       | 332ms           |
| Sending packets 17-20 | 0ms        | 332ms           |
| Arrival packets 17-20 | 18ms       | 350ms           |
| ACK of 20th packet    | 15ms       | 365ms           |

Arrival Time of the 20th packet: 365ms 